This file contains python code for convergence test under linear setting

In [1]:
import sys
sys.path.append("../mypkg")

In [2]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import seaborn as sns
import xarray as xr
from scipy.stats import pearsonr

from easydict import EasyDict as edict
from tqdm import trange
from scipy.io import loadmat

plt.style.use('ggplot')
plt.rcParams["savefig.bbox"] = "tight"

In [3]:
import importlib
import hdf_utils.data_gen
importlib.reload(hdf_utils.data_gen)

<module 'hdf_utils.data_gen' from '/data/rajlab1/user_data/jin/MyResearch/HDF_infer/notebooks/../mypkg/hdf_utils/data_gen.py'>

In [4]:
from constants import DATA_ROOT, RES_ROOT, FIG_ROOT

from hdf_utils.data_gen import gen_covs, gen_simu_psd
from utils.matrix import col_vec_fn, col_vec2mat_fn, conju_grad, svd_inverse
from utils.functions import logit_fn
from utils.misc import save_pkl
from splines import obt_bsp_basis_Rfn
from projection import euclidean_proj_l1ball
from optimization.one_step_opt import OneStepOpt

from penalties.scad_pen import SCAD
from penalties.base_pen import PenaltyBase
from models.logistic_model import LogisticModel
from models.linear_model import LinearModel


In [5]:
torch.set_default_tensor_type(torch.DoubleTensor)

## Param and fns

### Params

In [6]:
# freqs
ind_freq = np.linspace(1, 40, 40)

In [7]:
paras = edict()
paras.model = "linear"
paras.num_rep = 100
paras.n = 400 # num of data obs to be genareted

paras.npts = 40 # num of pts to evaluate X(s)
paras.d = 10 # num of ROIs
paras.q = 5 # num of other covariates
paras.sigma2 = 1 # variance of the error
paras.sel_idx = np.arange(2, paras.d) # M^c set
paras.stop_cv = 5e-4 # stop cv for convergence
paras.max_iter = 2000
paras.can_lams = [1e-1, 5e-1, 3e0, 2e1, 1e2]
paras.can_Rfcts = [1, 1.5, 2, 5, 10]


paras.bsp = edict()
paras.bsp.aknots_raw = np.linspace(0, 1, 20)
paras.bsp.iknots = paras.bsp.aknots_raw[1:-1]
paras.bsp.bknots = np.array([0, 1])
paras.bsp.ord = 4
paras.bsp.x = np.linspace(0, 1, paras.npts)
#paras.bsp.x = np.linspace(0, 1, paras.npts+1)[:-1]
paras.bsp.basis_mat = obt_bsp_basis_Rfn(paras.bsp.x, 
                                        paras.bsp.iknots, 
                                        paras.bsp.bknots, 
                                        paras.bsp.ord)
paras.bsp.N = paras.bsp.basis_mat.shape[1] # num of basis for bsp

paras.types_ = ["int", 2, 2, "c", "c"]
paras.alp_GT = np.array([5, 1, -2, 3, -4])

paras.Gam_GT = np.random.randn(paras.bsp.N, paras.d)*10
rm_idxs = np.arange(2, paras.d-4)
paras.Gam_GT[:, rm_idxs] = 0

paras.Rmin = 2*(np.linalg.norm(paras.Gam_GT/np.sqrt(paras.bsp.N), axis=0).sum() + np.abs(paras.alp_GT).sum())

paras.basis_mat = torch.DoubleTensor(paras.bsp.basis_mat) # npts x N

In [8]:
paras.save_dir = RES_ROOT/"linear_1"
if not paras.save_dir.exists():
    paras.save_dir.mkdir()
save_pkl(paras.save_dir/"paras.pkl", paras, is_force=True)

Save to /data/rajlab1/user_data/jin/MyResearch/HDF_infer/notebooks/../mypkg/../results/linear_1/paras.pkl


### Fns

In [9]:
def gen_simu_data_all(seed):
    np.random.seed(seed)
    # simulated PSD
    assert len(paras.types_) == paras.q
    assert len(paras.alp_GT) == paras.q
    
    simu_psd = gen_simu_psd(paras.n, paras.d, 10)
    simu_covs = gen_covs(paras.n, paras.types_)
    
    # linear term and Y
    beta_mat = paras.bsp.basis_mat @ paras.Gam_GT
    beta_mat = np.asarray(beta_mat.T)
    int_part = np.sum(beta_mat * simu_psd[:, :, :], axis=1).mean(axis=1)
    cov_part = simu_covs @ paras.alp_GT 
    
    # linear term
    lin_term = cov_part + int_part
    
    # Y 
    rvs = np.random.rand(paras.n)
    Y = lin_term + np.random.randn(paras.n)*np.sqrt(paras.sigma2)
    
    # To torch
    X = torch.tensor(simu_psd) # n x d x npts
    Z = torch.tensor(simu_covs) # n x q
    Y = torch.tensor(Y)
    
    all_data = edict()
    all_data.X = X
    all_data.Y = Y
    all_data.Z = Z
    return all_data

## Simu

In [10]:
def run_fn(seed, lam, Rfct):
    np.random.seed(seed)
    torch.manual_seed(seed)
    
    alp_init = torch.tensor(paras.alp_GT) + torch.randn(paras.q)*10
    Gam_init = torch.tensor(paras.Gam_GT) + torch.randn(paras.bsp.N, paras.d)*10
    theta_init = torch.cat([alp_init, col_vec_fn(Gam_init)/np.sqrt(paras.bsp.N)])
    rhok_init = torch.randn(paras.d*paras.bsp.N)
    last_Gamk = 0
    last_rhok = 0
    last_thetak = 0
    
    cur_data = gen_simu_data_all(seed)
    model = LinearModel(Y=cur_data.Y, X=cur_data.X, Z=cur_data.Z, 
                        basis_mat=paras.basis_mat, sigma2=paras.sigma2)
    # 3e0
    pen = SCAD(lams=lam, a=3.7,  sel_idx=paras.sel_idx)
    
    for ix in trange(paras.max_iter):
        opt = OneStepOpt(Gamk=Gam_init, 
                      rhok=rhok_init, 
                      theta_init=theta_init, 
                      alpha=0.9, beta=1, model=model, penalty=pen, 
                      q=paras.q, NR_eps=1e-5, NR_maxit=100, R=paras.Rmin*Rfct)
        opt()
        Gam_init = opt.Gamk
        rhok_init = opt.rhok
        theta_init = opt.thetak
        
        
        # converge cv
        Gam_diff = opt.Gamk- last_Gamk
        Gam_diff_norm = torch.norm(Gam_diff)/torch.norm(opt.Gamk)
        
        theta_diff = opt.thetak - last_thetak
        theta_diff_norm = torch.norm(theta_diff)/torch.norm(opt.thetak)
        
        Gam_theta_diff = opt.Gamk - col_vec2mat_fn(opt.thetak[paras.q:], nrow=paras.bsp.N)*np.sqrt(paras.bsp.N)
        Gam_theta_diff_norm = torch.norm(Gam_theta_diff)/torch.norm(opt.Gamk)
        
        stop_v = np.max([Gam_diff_norm.item(), theta_diff_norm.item(), Gam_theta_diff_norm.item()])
        if stop_v < paras.stop_cv:
            break
            
        last_Gamk = opt.Gamk
        last_rhok = opt.rhok
        last_thetak = opt.thetak
    
    if ix == (paras.max_iter-1):
        print(f"The optimization under seed {seed} may not converge with stop value {stop_v:.3f}")
    return opt, ix == (paras.max_iter-1)

In [11]:
for cur_lam in paras.can_lams:
    for cur_Rfct in paras.can_Rfcts:
        fil_name = f"result_lam-{cur_lam*100:.0f}_Rfct-{cur_Rfct*10:.0f}_n-{paras.n:.0f}.pkl"
        opt_results = []
        for seed in range(paras.num_rep):
            print(cur_lam, cur_Rfct, seed)
            opt_result = run_fn(seed, lam=cur_lam, Rfct=cur_Rfct)
            opt_results.append([opt_result, seed])
        save_pkl(paras.save_dir/fil_name, opt_results)

0.1 1 0


 31%|█████████████████████████████████████████▌                                                                                            | 621/2000 [00:01<00:03, 438.23it/s]


0.1 1 1


 34%|█████████████████████████████████████████████▌                                                                                        | 680/2000 [00:01<00:03, 428.21it/s]


0.1 1 2


...

 32%|███████████████████████████████████████████▌                                                                                          | 650/2000 [00:01<00:02, 458.39it/s]


20.0 10 33


 36%|███████████████████████████████████████████████▉                                                                                      | 716/2000 [00:01<00:02, 456.56it/s]


20.0 10 34


 39%|███████████████████████████████████████████████████▊                                                                                  | 773/2000 [00:01<00:02, 460.11it/s]
IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

